In [4]:
import pandas as pd
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]
data = {}

for data_file in data_files:
    #remove .csv in the file name for the variable name
    df_name = data_file[:-4]
    data[df_name] = pd.read_csv('../resources/' + data_file)

print(data.keys())

dict_keys(['ap_2010', 'class_size', 'demographics', 'graduation', 'hs_directory', 'sat_results'])


In [5]:
#make copy pointer to the sat dataframe
sat = data['sat_results']

print(sat.head())

for key,value in data.items():
    print('Data set name {}'.format(key))
    print(value.head())



      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                      7                             414                 401   
4                     44                             390                 433   

  SAT Writing Avg. Score  
0                    363  
1                    366  
2                    370  
3                    359  
4                    38

In [6]:
all_survey = pd.read_csv('../resources//survey_all.txt', delimiter='\t',
                         encoding='windows-1252')

print(all_survey.iloc[0])
d75_survey = pd.read_csv('../resources/survey_d75.txt', delimiter='\t',
                        encoding='windows-1252')
print(d75_survey.iloc[0])

survey = pd.concat([all_survey,d75_survey],axis=0)

print(survey.iloc[0])


dbn                                    01M015
bn                                       M015
schoolname          P.S. 015 Roberto Clemente
d75                                         0
studentssurveyed                           No
highschool                                  0
schooltype                  Elementary School
rr_s                                      NaN
rr_t                                       88
rr_p                                       60
N_s                                       NaN
N_t                                        22
N_p                                        90
nr_s                                        0
nr_t                                       25
nr_p                                      150
saf_p_11                                  8.5
com_p_11                                  7.6
eng_p_11                                  7.5
aca_p_11                                  7.8
saf_t_11                                  7.5
com_t_11                          

In [7]:
survey["DBN"] = survey["dbn"]

survey_fields = [
    "DBN", 
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey = survey.loc[:,survey_fields]
data["survey"] = survey

print(survey.head())

      DBN  rr_s  rr_t  rr_p    N_s   N_t    N_p  saf_p_11  com_p_11  eng_p_11  \
0  01M015   NaN    88    60    NaN  22.0   90.0       8.5       7.6       7.5   
1  01M019   NaN   100    60    NaN  34.0  161.0       8.4       7.6       7.6   
2  01M020   NaN    88    73    NaN  42.0  367.0       8.9       8.3       8.3   
3  01M034  89.0    73    50  145.0  29.0  151.0       8.8       8.2       8.0   
4  01M063   NaN   100    60    NaN  23.0   90.0       8.7       7.9       8.1   

      ...      eng_t_11  aca_t_11  saf_s_11  com_s_11  eng_s_11  aca_s_11  \
0     ...           7.6       7.9       NaN       NaN       NaN       NaN   
1     ...           8.9       9.1       NaN       NaN       NaN       NaN   
2     ...           6.8       7.5       NaN       NaN       NaN       NaN   
3     ...           6.8       7.8       6.2       5.9       6.5       7.4   
4     ...           7.8       8.1       NaN       NaN       NaN       NaN   

   saf_tot_11  com_tot_11  eng_tot_11  aca_tot_11 

In [9]:
def padded_csd(csd):
    csd_str = str(csd)
    if len(csd_str) == 1:
        csd_str = csd_str.zfill(2)
    return csd_str

def genDBN(row):
    return row['padded_csd'] + row['SCHOOL CODE']

hs_dir = data['hs_directory']

hs_dir['DBN'] = hs_dir['dbn']


class_size = data['class_size']
class_size['padded_csd'] = class_size['CSD'].apply(padded_csd)
class_size['DBN'] = class_size.apply(genDBN, axis=1)

print(class_size['DBN'].iloc[0:5])



0    01M015
1    01M015
2    01M015
3    01M015
4    01M015
Name: DBN, dtype: object


In [10]:
sat_results = data['sat_results']
string_to_num_cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score','SAT Writing Avg. Score']
sat_results['sat_score'] = [0]*len(sat_results)
for col in string_to_num_cols:
    sat_results[col] = pd.to_numeric(sat_results[col],errors='coerce')
    sat_results['sat_score'] = sat_results['sat_score'] + sat_results[col]

print(sat_results[string_to_num_cols].iloc[0])
print(sat_results['sat_score'].iloc[0])

SAT Math Avg. Score                404.0
SAT Critical Reading Avg. Score    355.0
SAT Writing Avg. Score             363.0
Name: 0, dtype: float64
1122.0


In [11]:
import re

def extract_lat(data):
    match = re.findall('\(.+\)', data)
    #splits the lat and long and remove first char '(' and last char from lat which is comma(,) and returns the string
    lat_long = match[0]
    return lat_long.split(' ')[0][1:-1]

hs_directory = data['hs_directory']

hs_directory['lat'] = hs_directory['Location 1'].apply(extract_lat)

print(hs_directory['lat'].iloc[0:5])

0     40.67029890700047
1      40.8276026690005
2    40.842414068000494
3     40.71067947100045
4    40.718810094000446
Name: lat, dtype: object


In [12]:
def extract_longitude(data):
    match = re.findall('\(.+\)', data)
    lat_long = match[0]
    #split lat and long and remove last char ')' from long then return it
    return lat_long.split(' ')[1][:-1]

hs_directory = data['hs_directory']

hs_directory['lon'] = hs_directory['Location 1'].apply(extract_longitude)
lat_lon = ['lat', 'lon']
for l in lat_lon:
    hs_directory[l] = pd.to_numeric(hs_directory[l], errors='coerce')

print(hs_directory.iloc[0:5])

      dbn                                        school_name       boro  \
0  17K548                Brooklyn School for Music & Theatre   Brooklyn   
1  09X543                   High School for Violin and Dance      Bronx   
2  09X327        Comprehensive Model School Project M.S. 327      Bronx   
3  02M280     Manhattan Early College School for Advertising  Manhattan   
4  28Q680  Queens Gateway to Health Sciences Secondary Sc...     Queens   

  building_code    phone_number    fax_number grade_span_min  grade_span_max  \
0          K440    718-230-6250  718-230-6262              9              12   
1          X400    718-842-0687  718-589-9849              9              12   
2          X240    718-294-8111  718-294-8109              6              12   
3          M520  718-935-3477             NaN              9              10   
4          Q695    718-969-3155  718-969-3552              6              12   

  expgrade_span_min  expgrade_span_max    ...      \
0              

In [13]:
class_size = data['class_size']
#drop rows where grade is different than '09-12' and program type diff than 'GEN ED'
program_type = class_size['PROGRAM TYPE'] != 'GEN ED'
grade = class_size['GRADE '] != '09-12'
program_type_grade = (program_type | grade)
class_size.drop(class_size[program_type_grade].index, inplace=True)

print(class_size.iloc[0:5])
                

     CSD BOROUGH SCHOOL CODE                                    SCHOOL NAME  \
225    1       M        M292  Henry Street School for International Studies   
226    1       M        M292  Henry Street School for International Studies   
227    1       M        M292  Henry Street School for International Studies   
228    1       M        M292  Henry Street School for International Studies   
229    1       M        M292  Henry Street School for International Studies   

    GRADE  PROGRAM TYPE CORE SUBJECT (MS CORE and 9-12 ONLY)  \
225  09-12       GEN ED                              ENGLISH   
226  09-12       GEN ED                              ENGLISH   
227  09-12       GEN ED                              ENGLISH   
228  09-12       GEN ED                              ENGLISH   
229  09-12       GEN ED                                 MATH   

    CORE COURSE (MS CORE and 9-12 ONLY) SERVICE CATEGORY(K-9* ONLY)  \
225                           English 9                           -  